1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные объявления с avito.ru в созданную БД (xpath/BS для парсинга на выбор)  
2) Написать функцию, которая производит поиск и выводит на экран объявления с ценой меньше введенной суммы  
*Написать функцию, которая будет добавлять в вашу базу данных только новые объявления

In [1]:
import requests 
from bs4 import BeautifulSoup
from pymongo import MongoClient
from collections import namedtuple
import time

In [2]:
Entry = namedtuple('Entry', ['id', 'title', 'price', 'href'])


# Возвращает результат запроса к сайту
def html_from_url(url, params = None, timeout = 50):
    msg = ''
    session = requests.Session()
    response = requests.Response()
    try:
        headers = {'accept': '*/&*',
                   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                                 '(KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
        if params is None:
            response = requests.get(url,headers=headers, timeout=timeout)
        else:
            response = requests.get(url,headers=headers,params=params, timeout=timeout)
        print(response.url)
    except requests.exceptions.ConnectionError:
        msg = 'Проблемы с интернет соединением'
    except requests.RequestException as e:
        msg = str(e)
    return response.text, response.status_code, msg

def parse_avito(url, category_url, page = 1):
    news_url = url+'/'+category_url
    result = []
    html_doc, status_code, msg = html_from_url(avito_url + category_url, {'p':f'{page}'}) 
    if status_code != 200:
        print(f'Ошибка загрузки страницы {news_url} : {msg}')
    else:
            soup = BeautifulSoup(html_doc, 'html.parser')    
            items = soup.findAll('div', {'class': 'item'})
            for item in items:
                id = item.attrs['id']
                descr = item.find('a', {'itemprop':'url'})
                href = avito_url + descr.attrs['href']          
                title = descr.attrs['title']
                try:
                    price = item.find('span', {'itemprop':'price'}).text
                except AttributeError:
                    price= item.find('span', {'class':'price-value'}).text
                try:
                    price = int(price.strip().replace(' ', '').replace('₽', ''))
                except ValueError:
                    price = None
                
                result.append(Entry(id, title, price, href))
    return result
#вставляет информацию об объявлении, если ранее объявления с таким идентификатором не было в базе данных
def insert_items(avito_db, items):
    for item in items:
        if avito_db.count_documents({ '_id': item.id}) == 0:
            avito_db.insert_one( {
                                '_id':item.id,
                                'title':item.title,
                                'price':item.price,
                                'href':item.href
                            })
#поиск объявлений с ценой не менее указанной
def item_search(price,collection):
    return collection.find( {'price': {'$gte': price} } )

In [3]:
MIN_PAGE = 9
MAX_PAGE = 9
avito_url = 'https://www.avito.ru'
category_url = '/perm/bytovaya_elektronika/'

client = MongoClient('mongodb://127.0.0.1:27017')
db = client['test']
avito_db = db.test
#test_db.drop()

for page in range(MIN_PAGE,MAX_PAGE + 1):
    items = parse_avito(avito_url, category_url, page) 
    insert_items(avito_db, items)
    if page !=MAX_PAGE + 1:
        time.sleep( 50 )

https://www.avito.ru/perm/bytovaya_elektronika?p=9


In [4]:
MIN_PRICE = 30000
for item in item_search(MIN_PRICE, avito_db):
    print(f'{item["title"]} стоит {item["price"]}\nСсылка: {item["href"]}\n')

Ноутбук Lenovo Yoga 530 с сенсорным экраном новый в Перми стоит 34500
Ссылка: https://www.avito.ru/perm/noutbuki/noutbuk_lenovo_yoga_530_s_sensornym_ekranom_novyy_1490123641

iPhone 8 Plus в Перми стоит 41990
Ссылка: https://www.avito.ru/perm/telefony/iphone_8_plus_968264057

iPhone Xs 64gb space grey в Перми стоит 72790
Ссылка: https://www.avito.ru/perm/telefony/iphone_xs_64gb_space_grey_1765028880

iPhone X в Перми стоит 49990
Ссылка: https://www.avito.ru/perm/telefony/iphone_x_1188477691

iPhone 7 128GB Gold Новый в Перми стоит 38490
Ссылка: https://www.avito.ru/perm/telefony/iphone_7_128gb_gold_novyy_971228856

iPhone 7 Plus 128GB Jet Black Новый в Перми стоит 43990
Ссылка: https://www.avito.ru/perm/telefony/iphone_7_plus_128gb_jet_black_novyy_991441944

iPhone 7 128GB (Розовое золото) Новый в Перми стоит 38490
Ссылка: https://www.avito.ru/perm/telefony/iphone_7_128gb_rozovoe_zoloto_novyy_891111559

iPhone 7 32GB Gold Новый в Перми стоит 30990
Ссылка: https://www.avito.ru/perm/tele